# ETL-Helper

*A little helper to help find the probable title*

In [1]:
#CORRECTION FOR TITLES
#!pip install PyYAML
#!pip install fold_to_ascii

In [2]:
import MungingOps as tt
import pandas as pd
import warnings

In [3]:
# Flexibility, rapid responce:
# This is not an oraculus, you not magicaly obtain the responce
# Manipulate the number to manipulate the result
#
#
# Flexibility, Razonable explication:
# Flexibility refers to the range of bad response can be accepted;
# this range is a quasy fuzzy response, not a binary logic
# then the Jaccard Distance say this string is most ok or not ok,
# traditionaly the jaccard distance in 0.8 is acceptable, but
# sometimes is necesary more low number to obtain a responce
#
Flexibility = 0.75
#
warnings.filterwarnings('ignore')

### Load the Dictionary

this file contains the relation of malformed tiles and their respective right titles

In [4]:
RAW = pd.read_excel('DictionaryTitles.xlsx')

Drop nulls

In [5]:
try:
    RAW.dropna(thresh=1, inplace=True)
except: None
try:
    RAW.dropna(thresh=2, inplace=True)
except: None


Work Safe with copy of Data

In [6]:
df=RAW

Load the Exceptions to evaluate

In [7]:
EF = pd.read_excel('ExceptionTitles.xlsx')
ExceptionTitles = EF.copy()
# Cut the dataset here to test
#
#
#
# PLEASE - IN PRODUCTION: - NO -
#
#
#
# ExceptionTitles=ExceptionTitles.head(10)

Drop Nulls

In [8]:
ExceptionTitles.dropna(inplace=True)

Clean the text to common english (130 ascii non unicoide) characters without symbols

In [9]:
chng=[]
for i in ExceptionTitles['Exceptions']:
    chng.append( tt.ExtraWhite(str(i).replace('/','').replace(':','').replace('&','').replace('*','').replace('-','').replace(',','').replace('.','').replace(' ','')).replace('"','').lower() )
ExceptionTitles['ExceptionsVector'] = chng
del chng

In [10]:
# Explore the changes in data
#
#
#
# PLEASE - IN PRODUCTION: - NO -
#
#
#
#ExceptionTitles.head()

In [11]:
chng=[]
for i in df['GP_Title']:
    chng.append( tt.ExtraWhite(str(i).replace('/','').replace(':','').replace('&','').replace('*','').replace('-','').replace(',','').replace('.','').replace(' ','')).replace('"','').lower() )
df['GP_Title'] = chng
del chng

In [12]:
# Explore the changes in data
#
#
#
# PLEASE - IN PRODUCTION: - NO -
#
#
#
#df.head()

Calculate the Jaccard Distance by each Exception for each GP_Title

In [13]:
# Calculate the Jaccard Distance by each Exception for each GP_Title
# Select the maximum Jaccard
# if Jaccard is > Flexibility the make a Suggestion
# si no pues no -0_o-
#
Suggested={}
Correction={}
Correction2={}
#
for i in ExceptionTitles['ExceptionsVector']:
    Comparison={}
    for j in df['GP_Title']:
        Comparison[j] = (tt.JaccardDistance(i,j))
        JaccardValues = max(zip(Comparison.values(),Comparison.keys()))
        MaxJaccard = JaccardValues[1]
    if JaccardValues[0] > Flexibility:
        Suggested[i] = JaccardValues[1] 
        Correction[i] = df[ df['GP_Title'] == MaxJaccard ]['Title'].values[0]
        Correction2[i] = df[ df['GP_Title'] == MaxJaccard ]['TitleID'].values[0]
    else:
        Suggested[i] = None
        Correction[i] = None
        Correction2[i] = None

In [14]:
# See the results
#
#
#
# PLEASE - IN PRODUCTION: - NO -
#
#
#
print(Suggested.values())
print(Correction.values())
print(Correction2.values())

dict_values([None, 'ciitreasuremd01110l', None, None, 'mightycashxtrareeljicai', 'tiger9princessofthedragonking', 'tiger9princessofthedragonking', 'xianghaoyunlonghaoyunlong', None, None])
dict_values([None, 'Treasure', None, None, 'Ji Cai', 'Tiger', 'Tiger', 'Xiang', None, None])
dict_values([None, 1246, None, None, 66, 488, 488, 369, None, None])


In [15]:
# Add the results to the dataframe
#
ExceptionTitles['Suggested'] = Suggested.values()
ExceptionTitles['Title'] = Correction.values()
ExceptionTitles['TitleID'] = Correction2.values()

Delete the unnecesary working data

In [16]:
# Delete the unnecesary working data
#
del df
ExceptionTitles.pop('ExceptionsVector')
ExceptionTitles

,Exceptions,Suggested,Title,TitleID
0,10 Cr Multiplier/,None,None,NaN
1,10 Credit Multipl,ciitreasuremd01110l,Treasure,1246.0
2,10Cr Multiplier/2,None,None,NaN
3,5 Credit Multipli,None,None,NaN
4,Mighty Cash Xtra Reel,mightycashxtrareeljicai,Ji Cai,66.0
5,9 Princes If The Dragon King,tiger9princessofthedragonking,Tiger,488.0
6,9 Princes Of The Dragon King,tiger9princessofthedragonking,Tiger,488.0
7,*Ji - Hao Yun Long,xianghaoyunlonghaoyunlong,Xiang,369.0
8,.01VARIANCE,None,None,NaN
9,.02VARIANCE,None,None,NaN


Save to original file

In [17]:
# Save to original file
# Enable in prod
ExceptionTitles.to_excel('ExceptionTitles.xlsx')